# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [51]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [52]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [53]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': ')L G ETSMLA SKONC/RLEATA(BDGENAQS',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'iilcffoa',
 'close': None,
 'closeTime': None,
 'closeSource': 'fcifolai',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 136.45,
 'latestSource': 'IEX real time price',
 'latestTime': '12:57:26 PM',
 'latestUpdate': 1672855569829,
 'latestVolume': None,
 'iexRealtimePrice': 136.18,
 'iexRealtimeSize': 311,
 'iexLastUpdated': 1682278133505,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 138.91,
 'previousVolume': 109840895,
 'change': 0.88,
 'changePercent': 0.00628,
 'volume': None,
 'iexMarketPercent': 0.017707740807690258,
 'iexVolume': 80

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [54]:
price = data['latestPrice']
peRatio = data['peRatio']
print(price)
print(peRatio)

136.45
37.05


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [55]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [56]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'n/a'
                ],
                index=my_columns
            ),
            ignore_index=True
        )

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [57]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe.head(50)
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,43.500,2.65,n/a
1,NLOK,22.780,4.4,n/a
2,AIV,5.051,4.86,n/a
3,UNM,23.950,5.18,n/a
4,BIO,616.410,5.27,n/a
5,AFL,46.858,7.54,n/a
6,EBAY,58.000,8,n/a
7,ALL,114.596,8.11,n/a
8,MET,51.074,8.31,n/a
9,CPB,48.710,8.32,n/a


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [58]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [59]:
portfolio_input()

Enter the value of your portfolio: 1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [60]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[row, 'Price'])
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,43.500,2.65,459
1,NLOK,22.780,4.4,877
2,AIV,5.051,4.86,3959
3,UNM,23.950,5.18,835
4,BIO,616.410,5.27,32
5,AFL,46.858,7.54,426
6,EBAY,58.000,8,344
7,ALL,114.596,8.11,174
8,MET,51.074,8.31,391
9,CPB,48.710,8.32,410


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [61]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data[symbol]['advanced-stats'])

# Price-to-earnings ratio
peRatio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pbRatio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
psRatio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev / ebitda
print(ev)

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = ev / gross_profit

2287957635072


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [87]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV-to-EBITDA Ratio',
    'EV-to-EBITDA Percentile',
    'EV-to-Gross Profit Ratio',
    'EV-to_Gross Profit Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for row in rv_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        ev = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        
        try:
            ev_to_ebitda = ev / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN    
            
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_gross_profit = ev / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN   

        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'n/a',
                    data[symbol]['quote']['peRatio'],
                    'n/a',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'n/a',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'n/a',
                    ev_to_ebitda,
                    'n/a',
                    ev_to_gross_profit,
                    'n/a',
                    'n/a'
                ],
                index = rv_columns
            ),
            ignore_index = True
        )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score
0,A,127.630,n/a,55.55,n/a,7.79,n/a,7,n/a,33.848206,n/a,13.538257,n/a,n/a
1,AAL,17.326,n/a,-1.2,n/a,-1.85,n/a,0.4225,n/a,-6.417391,n/a,1.433452,n/a,n/a
2,AAP,159.380,n/a,23.07,n/a,2.74,n/a,1.05,n/a,10.747832,n/a,2.333944,n/a,n/a
3,AAPL,138.500,n/a,38.07,n/a,34.26,n/a,7.83,n/a,26.896695,n/a,19.863275,n/a,n/a
4,ABBV,106.240,n/a,22.85,n/a,12.2,n/a,4.47,n/a,15.922213,n/a,9.355693,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.560,n/a,32.02,n/a,-4.1,n/a,5.77,n/a,22.761498,n/a,9.678948,n/a,n/a
501,ZBH,163.790,n/a,-223.3,n/a,2.78,n/a,4.57,n/a,26.264953,n/a,7.764203,n/a,n/a
502,ZBRA,424.500,n/a,47.59,n/a,11.34,n/a,5.01,n/a,32.073544,n/a,11.957014,n/a,n/a
503,ZION,47.200,n/a,20.57,n/a,1.04,n/a,2.52,n/a,11.037483,n/a,2.780753,n/a,n/a


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [88]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score
40,AON,211.540,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
71,BRK.B,239.980,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
118,CTL,11.000,n/a,10.16,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
136,DISCK,34.000,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
165,ETFC,50.160,n/a,14.76,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
168,EVRG,56.690,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
190,FOX,30.950,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
192,FRC,154.710,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
204,GOOG,1990.660,n/a,None,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a
326,MYL,16.130,n/a,32.19,n/a,None,n/a,None,n/a,NaN,n/a,NaN,n/a,n/a


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [89]:
numeric_cols = [
                'Price-to-Earnings Ratio',
                'Price-to-Book Ratio',
                'Price-to-Sales Ratio',
                'EV-to-EBITDA Ratio',
                'EV-to-Gross Profit Ratio'
]

for column in numeric_cols:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [90]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [91]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV-to-EBITDA Ratio': 'EV-to-EBITDA Percentile',
    'EV-to-Gross Profit Ratio': 'EV-to_Gross Profit Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric],
                                                       rv_dataframe.loc[row, metric])
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score
0,A,127.630,n/a,55.55,86.5347,7.79,73.8614,7.0000,79.802,33.848206,85.3465,13.538257,80.5941,n/a
1,AAL,17.326,n/a,-1.20,15.2475,-1.85,4.75248,0.4225,3.36634,-6.417391,2.9703,1.433452,5.74257,n/a
2,AAP,159.380,n/a,23.07,46.9307,2.74,38.3168,1.0500,14.3564,10.747832,23.9604,2.333944,11.2871,n/a
3,AAPL,138.500,n/a,38.07,75.0495,34.26,96.2376,7.8300,83.3663,26.896695,79.0099,19.863275,93.6634,n/a
4,ABBV,106.240,n/a,22.85,46.5347,12.20,84.5545,4.4700,62.7723,15.922213,48.5149,9.355693,64.3564,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.560,n/a,32.02,65.3465,-4.10,4.35644,5.7700,74.4554,22.761498,70.6931,9.678948,66.5347,n/a
501,ZBH,163.790,n/a,-223.30,2.57426,2.78,38.6139,4.5700,64.1584,26.264953,77.6238,7.764203,50.6931,n/a
502,ZBRA,424.500,n/a,47.59,82.1782,11.34,83.9604,5.0100,70.5941,32.073544,83.7624,11.957014,73.6634,n/a
503,ZION,47.200,n/a,20.57,42.5743,1.04,10.9901,2.5200,40.396,11.037483,26.7327,2.780753,14.0594,n/a


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [92]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score
0,A,127.630,n/a,55.55,86.5347,7.79,73.8614,7.0000,79.802,33.848206,85.3465,13.538257,80.5941,81.2277
1,AAL,17.326,n/a,-1.20,15.2475,-1.85,4.75248,0.4225,3.36634,-6.417391,2.9703,1.433452,5.74257,6.41584
2,AAP,159.380,n/a,23.07,46.9307,2.74,38.3168,1.0500,14.3564,10.747832,23.9604,2.333944,11.2871,26.9703
3,AAPL,138.500,n/a,38.07,75.0495,34.26,96.2376,7.8300,83.3663,26.896695,79.0099,19.863275,93.6634,85.4653
4,ABBV,106.240,n/a,22.85,46.5347,12.20,84.5545,4.4700,62.7723,15.922213,48.5149,9.355693,64.3564,61.3465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.560,n/a,32.02,65.3465,-4.10,4.35644,5.7700,74.4554,22.761498,70.6931,9.678948,66.5347,56.2772
501,ZBH,163.790,n/a,-223.30,2.57426,2.78,38.6139,4.5700,64.1584,26.264953,77.6238,7.764203,50.6931,46.7327
502,ZBRA,424.500,n/a,47.59,82.1782,11.34,83.9604,5.0100,70.5941,32.073544,83.7624,11.957014,73.6634,78.8317
503,ZION,47.200,n/a,20.57,42.5743,1.04,10.9901,2.5200,40.396,11.037483,26.7327,2.780753,14.0594,26.9505


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [93]:
# Remove Price-to-Earnings Ratios < 0, since companies with a negative earnings are not sound investments
rv_dataframe = rv_dataframe[rv_dataframe['Price-to-Earnings Ratio'] > 0]

rv_dataframe.sort_values('RV Score', ascending=True, inplace=True, ignore_index=True)
rv_dataframe = rv_dataframe.head(50)
rv_dataframe

<ipython-input-93-cb4413c4f0e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=True, inplace=True, ignore_index=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score
0,UNM,24.622,n/a,5.13,19.2079,0.4524,5.34653,0.4070,2.9703,2.651620,3.56436,0.399814,0.39604,6.29703
1,MET,50.950,n/a,8.29,20.396,0.6100,6.13861,0.6886,6.33663,4.392963,4.35644,0.647650,1.18812,7.68317
2,HIG,50.100,n/a,10.46,22.3762,0.9965,9.10891,0.8491,9.10891,4.200241,3.9604,0.849820,2.17822,9.34653
3,ALL,113.690,n/a,8.03,19.802,1.3100,16.5347,0.7757,7.92079,2.534620,3.36634,0.727514,1.58416,9.84158
4,LNC,49.930,n/a,12.35,25.5446,0.4162,5.14851,0.5240,4.15842,8.434205,14.2574,0.488488,0.594059,9.94059
5,PFG,52.860,n/a,11.65,23.9604,0.9000,8.31683,0.9552,10.9901,7.576347,10.099,0.906095,2.57426,11.1881
6,CVS,73.882,n/a,12.22,24.9505,1.4200,18.6139,0.3655,2.37624,7.642159,10.495,1.398539,5.34653,12.3564
7,HPQ,25.744,n/a,12.62,25.9406,-14.5600,2.77228,0.5650,4.75248,7.982210,11.8812,3.159768,17.4257,12.5545
8,C,60.400,n/a,12.17,24.7525,0.7169,6.33663,1.3000,20.6931,6.686235,6.73267,1.615586,6.93069,13.0891
9,AFL,47.475,n/a,7.65,19.604,1.0245,10.297,1.5100,24.0594,6.141981,5.54455,1.447783,5.94059,13.0891


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [96]:
portfolio_input()

Enter the value of your portfolio: 2500000


In [98]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for i in rv_dataframe.index:
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[i, 'Price'])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV-to-EBITDA Ratio,EV-to-EBITDA Percentile,EV-to-Gross Profit Ratio,EV-to_Gross Profit Percentile,RV Score
0,UNM,24.622,2030,5.13,19.2079,0.4524,5.34653,0.4070,2.9703,2.651620,3.56436,0.399814,0.39604,6.29703
1,MET,50.950,981,8.29,20.396,0.6100,6.13861,0.6886,6.33663,4.392963,4.35644,0.647650,1.18812,7.68317
2,HIG,50.100,998,10.46,22.3762,0.9965,9.10891,0.8491,9.10891,4.200241,3.9604,0.849820,2.17822,9.34653
3,ALL,113.690,439,8.03,19.802,1.3100,16.5347,0.7757,7.92079,2.534620,3.36634,0.727514,1.58416,9.84158
4,LNC,49.930,1001,12.35,25.5446,0.4162,5.14851,0.5240,4.15842,8.434205,14.2574,0.488488,0.594059,9.94059
5,PFG,52.860,945,11.65,23.9604,0.9000,8.31683,0.9552,10.9901,7.576347,10.099,0.906095,2.57426,11.1881
6,CVS,73.882,676,12.22,24.9505,1.4200,18.6139,0.3655,2.37624,7.642159,10.495,1.398539,5.34653,12.3564
7,HPQ,25.744,1942,12.62,25.9406,-14.5600,2.77228,0.5650,4.75248,7.982210,11.8812,3.159768,17.4257,12.5545
8,C,60.400,827,12.17,24.7525,0.7169,6.33663,1.3000,20.6931,6.686235,6.73267,1.615586,6.93069,13.0891
9,AFL,47.475,1053,7.65,19.604,1.0245,10.297,1.5100,24.0594,6.141981,5.54455,1.447783,5.94059,13.0891


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [125]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [126]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [127]:
column_formats = {
                    'A':['Ticker', string_template],
                    'B':['Price', dollar_template],
                    'C':['Number of Shares to Buy', integer_template],
                    'D':['Price-to-Earnings Ratio', percent_template],
                    'E':['PE Percentile', integer_template],
                    'F':['Price-to-Book Ratio', percent_template],
                    'G':['PB Percentile', integer_template],
                    'H':['Price-to-Sales Ratio', percent_template],
                    'I':['PS Percentile', integer_template],
                    'J':['EV-to-EBITDA Ratio', percent_template],
                    'K':['EV-to-EBITDA Percentile', integer_template],
                    'L':['EV-to-Gross Profit Ratio', percent_template],
                    'M':['EV-to_Gross Profit Percentile', integer_template],
                    'N':['RV Score', integer_template]
}

In [128]:
for column in column_formats.keys():
     writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
     writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])   

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [25]:
writer.save()